In [1]:
from tutorial_utils import magma_to_verilog_string, smt_to_smtlib_string

import hwtypes as hw
import magma as m

/home/donovick/work/peak-tutorial/env-peak-tutorial/lib/python3.10/site-packages/ast_tools/immutable_ast.py:10: UserWarning: /home/donovick/work/peak-tutorial/env-peak-tutorial/lib/python3.10/site-packages/ast_tools/immutable_ast.py generated for (3, 9)does not match system version (3, 10)
  warnings.warn(f"{__file__} generated for (3, 9)"


In [2]:
DataT = hw.BitVector[3]
Bit = hw.Bit

In [3]:
def match(value, match_dict, default): return DataT(2)

x = DataT(0b011)

y = match(
    x, 
    {
        '1x0': DataT(0),
        '11x': DataT(1),
    },
    default=DataT(2)
)

assert y == DataT(2)

In [4]:
def match(value, match_dict, default):
    if value[2] & ~value[0]: # matches 1x0
        return DataT(0)
    elif value[2] & value[1]: # matches 11x
        return DataT(1)
    else: # default
        return DataT(2)
    
y = match(
    x, 
    {
        '1x0': DataT(0),
        '11x': DataT(1),
    },
    default=DataT(2)
)

assert y == DataT(2)

In [6]:
def match(value, match_dict, default):
    if matches_string(value, '1x0'):
        return match_dict['1x0']
    elif matches_string(value, '11x'):
        return match_dict['11x']
    else: 
        return default

def matches_string(value, string):
    assert len(value) == len(string)
    matches = Bit(True)
    for i, c in enumerate(reversed(string)):
        if c == '1':
            matches &= value[i] # matches = matches & value[i]
        elif c == '0':
            matches &= ~value[i]
            
    return matches
        
y = match(
    x, 
    {
        '1x0': DataT(0),
        '11x': DataT(1),
    },
    default=DataT(2)
)

assert y == DataT(2)

In [7]:
def match(value, match_dict, default):
    if matches_string(value, '1x0'):
        return match_dict['1x0']
    elif matches_string(value, '11x'):
        return match_dict['11x']
    else: 
        return default

def match(value, match_dict, default):
    for key in match_dict:
        if matches_string(value, key):
            return match_dict[key]
    return default

y = match(
    x, 
    {
        '1x0': DataT(0),
        '11x': DataT(1),
    },
    default=DataT(2)
)

assert y == DataT(2)

In [11]:
# Works fully
def match(value, match_dict, default):
    for key in match_dict:
        if matches_string(value, key):
            return match_dict[key]
    return default

# Or this one more accurately
def match(value, match_dict, default):
    r_val = default
    for key in reversed(match_dict):
        r_val = matches_string(value, key).ite(
            match_dict[key],
            r_val
        )
    return r_val

y = match(
    x, 
    {
        '1x0': DataT(0),
        '11x': DataT(1),
    },
    default=DataT(2)
)

assert y == DataT(2)

In [20]:
# Alternative construction
def match(value, match_dict, default):
    if matches_string(value, '1x0'):
        return match_dict['1x0']
    elif matches_string(value, '11x'):
        return match_dict['11x']
    else: 
        return default

def match(value, match_dict, default):
    return matches_string(value, '1x0').ite(
        match_dict['1x0'],
        matches_string(value, '11x').ite(
            match_dict['11x'],
            default
        ),    
    )

def match(value, match_dict, default):
    r_val = default
    for key in reversed(match_dict):
        r_val = matches_string(value, key).ite(
            match_dict[key],
            r_val
        )
    return r_val

        
y = match(
    x, 
    {
        '1x0': DataT(0),
        '11x': DataT(1),
    },
    default=DataT(2)
)

assert y == DataT(2)

In [21]:
# Top Down
def tree_builder(data):
    if is_base_case(data):
        return genetate_base(data) 
    else:
        left_child = tree_builder(generate_left(data))
        right_child = tree_builder(generate_right(data))
        node = build_node(left_child, right_child)
        return node

def match(value, match_dict, default):
    def ite_builder(keys, idx):
        if len(keys) == idx:
            return default
        else:
            key = keys[idx]
            cond = matches_string(value, key)
            left = match_dict[key]
            right = ite_builder(keys, idx+1)
            node = cond.ite(left, right)
            return node
        
    return ite_builder(list(match_dict), 0)

y = match(
    x, 
    {
        '1x0': DataT(0),
        '11x': DataT(1),
    },
    default=DataT(2)
)

assert y == DataT(2)

In [22]:
def match(value, match_dict, default):
    r_val = default(value)
    for key in reversed(match_dict):
        r_val = matches_string(value, key).ite(
            match_dict[key](value),
            r_val
        )
    return r_val

def foo(x): return x

def default(x): return x + 2

y = match(
    x, 
    {
        '1x0': foo,
        '11x': lambda x: x + 1,
    },
    default=default,
)

assert y == x + 2